## 讀取工廠距離數據

In [1]:

import pandas as pd
from pulp import *

dist= pd.read_excel('https://raw.githubusercontent.com/ywchiu/HKPC/master/data/warehouse_city.xlsx')

dist= dist.set_index('Warehouse')


demand=[10000,20000,33000,9000,60000,2500,35000]
warehouse= dist.index
customers= dist.columns

keys= [(w,c) for w in warehouse for c in customers]

dist_dict= { (w,c): dist.loc[w,c] for w in warehouse for c in customers}


flows= LpVariable.dicts('flows',keys,cat='Binary')
open_w= LpVariable.dicts('open_w',warehouse,cat='Binary')

demand_dict= dict(zip(customers,demand))




## 距離數據優化

In [2]:
model= LpProblem('w_alloc',LpMinimize)

model+= lpSum([demand_dict[(c)]* flows[(w,c)]* dist_dict[(w,c)] for c in customers for w in warehouse])


for c in customers:
    model+= lpSum([flows[(w,c)] for w in warehouse])== 1
    

model+= lpSum([open_w[(w)] for w in warehouse])==3



for w in warehouse:
    model+= open_w[(w)]>= flows[(w,'city 1')]
    model+= open_w[(w)]>= flows[(w,'city 2')]
    model+= open_w[(w)]>= flows[(w,'city 3')]
    model+= open_w[(w)]>= flows[(w,'city 4')]
    model+= open_w[(w)]>= flows[(w,'city 5')]
    model+= open_w[(w)]>= flows[(w,'city 6')]
    model+= open_w[(w)]>= flows[(w,'city 7')]

model.solve()

for i in open_w:
    print(open_w[i],open_w[i].varValue)

flows_names= ['{} to {}'.format(w, c) for w in warehouse for c in customers]
flows_quantity= [flows[(w,c)].varValue for w in warehouse for c in customers]
flows_dict= dict(zip(flows_names,flows_quantity))

average_distance= value(model.objective)/sum(demand)
n_warehouses_opened= sum([open_w[(w)].varValue for w in warehouse])



open_w_city_1 0.0
open_w_city_2 1.0
open_w_city_3 1.0
open_w_city_4 0.0
open_w_city_5 1.0
open_w_city_6 0.0
open_w_city_7 0.0
